In [ ]:
#Created venv, installed/import required libraries

import pandas as pd
import numpy as np
from datetime import datetime

In [70]:
#Reading and creating DataFrame usign csv files.

customers= pd.read_csv("../data/customers.csv")
subscriptions= pd.read_csv("../data/subscriptions.csv")
events= pd.read_csv("../data/events.csv")

In [71]:
#Checking shape

print(customers.shape)
print(subscriptions.shape)
print(events.shape)

(1000, 5)
(941, 6)
(2411, 5)


In [72]:
#Checking columns names

print(customers.columns.tolist())
print(subscriptions.columns.tolist())
print(events.columns.tolist())

#Finding:
#customer_id is available in all there dataframes and name is same in all of them.
#No changes or normalisation needed for column names.

['customer_id', 'signup_date', 'segment', 'country', 'is_enterprise']
['subscription_id', 'customer_id', 'start_date', 'end_date', 'monthly_price', 'status']
['event_id', 'customer_id', 'event_type', 'event_date', 'source']


In [73]:
print(customers.head())
print(subscriptions.head())
print(events.head())

  customer_id signup_date     segment country  is_enterprise
0       C0001  2023-01-04  Enterprise      IN           True
1       C0002  2023-01-14         SMB      CA          False
2       C0003  2023-01-04         SMB      IN          False
3       C0004  2023-03-19         SMB      CA          False
4       C0005  2023-03-25         NaN      IN          False
  subscription_id customer_id  start_date    end_date  monthly_price    status
0          S00001       C0001  2023-01-10         NaN            499    active
1          S00002       C0002  2023-01-12         NaN             49    active
2          S00003       C0003  2023-01-08  2023-02-14            149  canceled
3          S00004       C0004  2023-01-21         NaN            149    active
4          S00005       C0005  2023-04-03         NaN             99    active
  event_id customer_id   event_type event_date    source
0  E000001       C0001       signup  3/10/2023   organic
1  E000002       C0001       signup  3/10/2023

In [74]:
#chekcing data types for each field

print(customers.dtypes)
print(subscriptions.dtypes)
print(events.dtypes)

# Findings: 
# Dates are stored as string. 
# Price it int but we should keep it as float in case there is any decimal value in future

customer_id       str
signup_date       str
segment           str
country           str
is_enterprise    bool
dtype: object
subscription_id      str
customer_id          str
start_date           str
end_date             str
monthly_price      int64
status               str
dtype: object
event_id       str
customer_id    str
event_type     str
event_date     str
source         str
dtype: object


In [75]:
#using info to have basic overview of the data

print(customers.info())
print(subscriptions.info())
print(events.info())

<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   customer_id    1000 non-null   str  
 1   signup_date    964 non-null    str  
 2   segment        757 non-null    str  
 3   country        1000 non-null   str  
 4   is_enterprise  1000 non-null   bool 
dtypes: bool(1), str(4)
memory usage: 32.4 KB
None
<class 'pandas.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   subscription_id  941 non-null    str  
 1   customer_id      941 non-null    str  
 2   start_date       941 non-null    str  
 3   end_date         223 non-null    str  
 4   monthly_price    941 non-null    int64
 5   status           941 non-null    str  
dtypes: int64(1), str(5)
memory usage: 44.2 KB
None
<class 'pandas.DataFrame'>
RangeIndex: 2411 entries, 0 to 2410
D

In [76]:
# Fixing data types for date column and monthly_price      

# changes in customers
customers["signup_date"] = pd.to_datetime(
    customers["signup_date"], 
    errors="coerce"
)

# changed in events
events["event_date"] = pd.to_datetime(
    events["event_date"], 
    errors="coerce"
)

# changes in subscriptions
subscriptions["start_date"] = pd.to_datetime(
    subscriptions["start_date"], 
    errors="coerce"
)

subscriptions["end_date"] = pd.to_datetime(
    subscriptions["end_date"], 
    errors="coerce"
)

subscriptions["monthly_price"]=subscriptions["monthly_price"].astype(float)

In [77]:
# checking if the primary keys in dataframes are unique or not.

print(customers["customer_id"].is_unique)
print(subscriptions["subscription_id"].is_unique)
print(events["event_id"].is_unique)

#Findings:
#All IDs are unique.

True
True
True


In [78]:
#Duplicate check row level

print(customers.duplicated().sum())
print(subscriptions.duplicated().sum())
print(events.duplicated().sum())

#Findings:
#no 2 rows in any dataframe are same.

0
0
0


In [79]:
#Checking if there is any customer id missing in subscription or event

print((~subscriptions["customer_id"].isin(customers["customer_id"])).sum())

print((~events["customer_id"].isin(customers["customer_id"])).sum())


0
0


In [ ]:
#Finding null values

print(customers.isna().sum())
print(subscriptions.isna().sum())
print(events.isna().sum())

#Findings:
#signup_date      36  -- customers
#segment          243 -- customers
#end_date         718 --subscription (This is fine as it is null for those who are active customers)


customer_id        0
signup_date       36
segment          243
country            0
is_enterprise      0
dtype: int64
subscription_id      0
customer_id          0
start_date           0
end_date           718
monthly_price        0
status               0
dtype: int64
event_id       0
customer_id    0
event_type     0
event_date     0
source         0
dtype: int64


In [ ]:
# Focusing on customers
# Fixing signup date using event dataframe.

first_signup= pd.DataFrame(events.loc[events['event_type']=="signup"].groupby("customer_id")["event_date"].min())
customers = customers.merge(first_signup, on="customer_id",how="left")
customers["signup_date"] = customers["signup_date"].fillna(customers["event_date"])

customers.drop("event_date",axis=1, inplace=True)

In [ ]:
#In the event data it is observed that we have duplicates based on customer_id, event_Type and event_date
#For example same customer_id signup event is captured with same date twice

#Removing duplicates where we have same customer_id, event_Type and event_date, will keep the frist (older one)

events= events.sort_values("event_id")

events = events.drop_duplicates(subset=["customer_id","event_type","event_date"],
                                keep = "first")

In [144]:
#Similar scenerio in subscription dataframe whee we have same customer as duplicate

subscriptions= subscriptions.sort_values("subscription_id")

subscriptions = subscriptions.drop_duplicates(subset=["customer_id","start_date","monthly_price","status"],
                                keep = "first")

In [ ]:
#segment check to find if we have any segment which are smb or mid-market where we have is_enterprise as True

customers.loc[customers["is_enterprise"]==True].groupby("segment")["segment"].count()

#For all the True values we have enterprise.

segment
Enterprise    250
Name: segment, dtype: int64

In [123]:
#For the missing segment where whe have is_enterprise = False, I am marking them as SMB
#due to non availability of other company based data and it is better to use SMB to avoid overfitting
#Even with the monthly_price in subscription I am unable to infer any choice
#Even with no subcription the customers in the datasets are enterise so with price it is not possible to find the correct segment.

customers.loc[(customers["is_enterprise"] == False) & (customers["segment"].isnull()),"segment"] = "SMB"



Data Load

In [ ]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

MYSQL_USER = ""
MYSQL_PASSWORD = ""     
MYSQL_HOST = ""
MYSQL_PORT = "3306"         
MYSQL_DB = ""

PASSWORD_ENC = quote_plus(MYSQL_PASSWORD)

engine = create_engine(
    f"mysql+mysqlconnector://{MYSQL_USER}:{PASSWORD_ENC}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
)

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print("DB connection OK:", result.fetchone())


DB connection OK: (1,)


In [ ]:
# Final Data load
customers.to_sql("customers_tb", engine, if_exists="append", index=False)
subscriptions.to_sql("subscriptions_tb", engine, if_exists="append", index=False)
events.to_sql("events_tb", engine, if_exists="append", index=False)

print("Data loade Done!")

Data loade Done!
